In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
import datetime
%matplotlib inline

In [2]:
df = pd.read_csv(r'D:\python_work\datasets\data_ab.csv')

In [3]:
df.head(3)

,Unnamed: 0,user_id,timestamp,group,landing_page,converted
0,0,851104,2021-01-21 22:11:48.556739,control,old_page,0
1,1,804228,2021-01-12 08:01:45.159739,control,old_page,0
2,2,661590,2021-01-11 16:55:06.154213,treatment,new_page,0


**Определение эксперимента**   
Мы разработали новую веб-страницу и хотим проверить ее влияние на конверсию покупок. Таким образом, мы делим наших пользователей поровну на 2 группы:  

1.Control: пользователь попадает на старую версию страницы  
2.Treatment: пользователь попадает на новую версию страницы  

Показатель, который мы хотим отслеживать:  

$$Perchase Conversion =\frac{Converted Users}{Exposed Users}$$


У нас есть данные о действиях / конверсии за 3 недели. Давайте определим эти термины:  
Действие: пользователь попадает в контрольную или исследуемую группу и видит соответствующую страницу впервые за время эксперимента.  
Конверсия: пользователь совершает покупку в течение 7 дней с момента первого действия.  

Каждая строка регистрируется, когда пользователь открывает веб-страницу.  

Timestamp: время, когда пользователь впервые открывает страницу  
Группа: контрольная/эксперементальная  
Landing_page: какую страницу они видят  
Конверсия: инициализировано на 0. Изменяется на 1, если пользователь совершает покупку в течение 7 дней с момента первого контакта.


In [4]:
# колонка "Unnamed: 0" не несет никакой информации(можно ее удалить)
df = df.drop('Unnamed: 0',axis=1)

In [5]:
# Проверим типы данных
df.dtypes

user_id          int64
timestamp       object
group           object
landing_page    object
converted        int64
dtype: object

In [6]:
#Проверим размер таблицы
df.shape

(294478, 5)

In [7]:
# Колонка 'timestamp' имеет тип 'object'. Переведем ее в формат даты
df.timestamp = pd.to_datetime(df['timestamp'])

In [8]:
df.dtypes

user_id                  int64
timestamp       datetime64[ns]
group                   object
landing_page            object
converted                int64
dtype: object

In [9]:
# Проверим количество пропущенных значений в данных
df.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [10]:
# Проверим диапазон дат в данных, коичество уникальных пользователей, страницы для сравнения, 
#соотношение пользователей в группах
start_time = df.timestamp.min()
end_time = df.timestamp.max() 
data_duration = (end_time - start_time).days

print(f"Number of unique users in experiment: {df['user_id'].nunique()}")
print(f"Data collected for {data_duration} days")
print(f"Landing pages to compare: {df['landing_page'].unique().tolist()}")
print(f"Percentage of users in control: {round(df[df['group']=='control'].shape[0] * 100 / df.shape[0])}%")

Number of unique users in experiment: 290584
Data collected for 21 days
Landing pages to compare: ['old_page', 'new_page']
Percentage of users in control: 50%


In [11]:
# Обшее количество записей в данных - 294478, а уникальных пользователей - 290584
# проверим пользователей, для которых имеется несколько записей
counter = df['user_id'].value_counts()
(counter > 1).value_counts()

False    286690
True       3894
Name: user_id, dtype: int64

In [12]:
counter[counter > 1].sample(1)

643619    2
Name: user_id, dtype: int64

In [13]:
sample = df[df['user_id'].isin([782843])]

In [14]:
sample

,user_id,timestamp,group,landing_page,converted
12120,782843,2021-01-04 12:22:20.249958,treatment,new_page,0
267975,782843,2021-01-14 13:58:20.935978,treatment,old_page,1


Для 3894 (1.35 %) user_ids есть записи взаимодействия со старой и новой страницей(их можно удалить)

In [15]:
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])

In [16]:
df = df.merge(valid_users, on=['user_id'])
print(f"Percentage of users in control: {round(df[df['group']=='control'].shape[0] * 100 / df.shape[0])}%")

Percentage of users in control: 50%


In [17]:
df.landing_page.value_counts()

new_page    143397
old_page    143293
Name: landing_page, dtype: int64

In [18]:
# Видим,что после преобразования процентное соотношение пользовательй взаимодействовавщих со старой и новой страницей
#не изменилось.Однако, есть небольшая разница в количестве пользователей
df.landing_page.value_counts()[0] - df.landing_page.value_counts()[1]

104

In [19]:
# Удалим из данных 104 случайных пользователя, чтобы уравнять количество в контрольной и исследуемой группах
df = df.drop(df[df.landing_page == 'new_page'].sample(104).index)

In [20]:
df.landing_page.value_counts()

old_page    143293
new_page    143293
Name: landing_page, dtype: int64

In [21]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2021-01-21 22:11:48.556739,control,old_page,0
1,804228,2021-01-12 08:01:45.159739,control,old_page,0
2,661590,2021-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2021-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2021-01-21 01:52:26.210827,control,old_page,1


In [22]:
# Добавим столбец 'week', чтобы просматривать данные как во время эксперимента
df['week'] = df.timestamp.dt.isocalendar().week

In [23]:
df.head(1)

,user_id,timestamp,group,landing_page,converted,week
0,851104,2021-01-21 22:11:48.556739,control,old_page,0,3


In [24]:
df['week'] = np.where(df['week'] != 53, df['week'], 1)

In [25]:
df['week'].value_counts()

1    109953
2     91029
3     85604
Name: week, dtype: int64

Мы тестируем целевую страницу и хотим посмотреть, будет ли она лучше.  
Ho - связи между целевой страницей, которую получают посетители, и их коэффициентом конверсии не существует.  
H1: существует связь между целевой страницей, которую получают посетители, и их коэффициентом конверсии.

Хи - квадрат тест

In [26]:
NUM_WEEKS = 3 # Можно изменять для получения результатов в разные моменты времени
experiment_data = df[df['week'] <= NUM_WEEKS]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round(treatment_conversion_perc - control_conversion_perc, 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.874%
Control Conversion Rate: 12.017%
Lift = -0.143%


In [27]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted
observed_table = np.array([[control_converted, control_non_converted], 
                              [treatment_converted, treatment_non_converted]])

In [28]:
observed_table

array([[ 17220, 126073],
       [ 17015, 126278]], dtype=int64)

In [30]:
chi, p_value, deg_of_fr, expected_table = chi2_contingency(observed_table, correction=False)

In [31]:
chi, p_value

(1.3940788252423906, 0.23771748963425166)

Вывод:  
Мы поучили p-value = 0.24(что больше 0.05).В этом случае значение хи-квадрат должно быть равно или превышать 3,84. Тогда результаты будут статистически значимыми. Поскольку 1,39 меньше 3,84, мои результаты статистически не отличаются. Нет никакой связи между тем, какую версию целевой страницы посетитель получает, и коэффициентом конверсии со статистической значимостью.